# Check for successful execution

In [1]:
distances = [15, 35]
doubleplanes = [12, 30]
energies = [200, 600, 1000]
neutrons = [1]
physicss = ["bert", "bic", "inclxx"]

In [2]:
import pandas as pd
import numpy as np
import joblib
import os
import ROOT

ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


In [3]:
def processed_events(distance, doubleplane, energy, neutron, physics, step):
    filename = f"output/{distance}m_{doubleplane}dp_{energy}AMeV_{neutron}n_{physics}.{step}.root"
    if os.path.isfile(filename):
        try:
            tfile = ROOT.TFile.Open(filename)
            ttree = tfile.Get("evt")
            num_events = int(ttree.GetEntries())
            return (filename, num_events)
        except:
            pass
    return (filename, 0)

In [4]:
evts = pd.DataFrame(
    joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
        joblib.delayed(processed_events)(
            distance=distance, doubleplane=doubleplane, energy=energy, neutron=neutron, physics=physics, step="simu",
        )
        for distance in distances
        for doubleplane in doubleplanes
        for energy in energies
        for neutron in neutrons
        for physics in physicss
    )
)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  36 | elapsed:    3.0s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    7.9s finished


In [5]:
pd.options.display.max_rows = 5
evts.columns = ["File", "Events"]
evts.sort_values(by=["Events", "File"], ascending=[True, True], inplace=True)
evts.style.hide_index().format({"Events": "{:d}"})
display(evts)

,File,Events
6,output/15m_12dp_1000AMeV_1n_bert.simu.root,100000
7,output/15m_12dp_1000AMeV_1n_bic.simu.root,100000
...,...,...
31,output/35m_30dp_600AMeV_1n_bic.simu.root,100000
32,output/35m_30dp_600AMeV_1n_inclxx.simu.root,100000


In [6]:
evts = pd.DataFrame(
    joblib.Parallel(n_jobs=-1, backend="multiprocessing", verbose=1)(
        joblib.delayed(processed_events)(
            distance=distance, doubleplane=doubleplane, energy=energy, neutron=neutron, physics=physics, step="digi",
        )
        for distance in distances
        for doubleplane in doubleplanes
        for energy in energies
        for neutron in neutrons
        for physics in physicss
    )
)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  36 | elapsed:    2.6s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.9s finished


In [7]:
pd.options.display.max_rows = 5
evts.columns = ["File", "Events"]
evts.sort_values(by=["Events", "File"], ascending=[True, True], inplace=True)
evts.style.hide_index().format({"Events": "{:d}"})
display(evts)

,File,Events
6,output/15m_12dp_1000AMeV_1n_bert.digi.root,100000
7,output/15m_12dp_1000AMeV_1n_bic.digi.root,100000
...,...,...
31,output/35m_30dp_600AMeV_1n_bic.digi.root,100000
32,output/35m_30dp_600AMeV_1n_inclxx.digi.root,100000
